# Recitation Week 3

This week we will review pattern matching and inductive data structures, and get some practice using them.

## Pattern Matching

Pattern matching is a control structure we will use many times in this class, especially when dealing with inductive structures.
Let's see some of the ways we can use pattern matching.

In [12]:
// What is the output of this code block? (Try to figure it out without running the code block)
// How could we modify this code so it outputs "The Pair of Numbers num1 and num2" when myList has 2 elements?
val myList = 131 :: 3155 :: Nil
myList match {
    case Nil => "No Numbers"
    case h :: Nil => s"The Number $h"
    // Begin solution
    case h :: t :: Nil => s"The Couple Numbers $h and $t"
    // End solution
    case h :: _ => "Multiple Numbers, with head " + h
}

myList: List[Int] = List(131, 3155)
res11_1: String = "The Couple Numbers 131 and 3155"

### More advanced pattern matching (extra)

Pattern matching also has other features beyond those shown above.
Some that might come in handy later in the course are shown below.
In particular, pattern guards and matching multiple things in the same case.
It is also possible to match on the type of a variable if the type of the value being matched on is unknown.

Matching multiple patterns in a single case is done by placing a single vertical bar (`|`) between each pattern.  If any of the patterns is matched then the branch is taken. See line 6 in the code below.

Pattern guards are used to make a case more specific then a pattern allows.  
To use a guard we add the keyword `if` after the pattern and then put a boolean condition. If the pattern matches and the condition evaluates to `true`, then the case is taken.  Otherwise, the case is skipped.
One example is on line 7 in the code below.  

In [2]:
val numThings = 3
numThings match {
    case 0 => "Nothing here"
    case 1 => "One thing here"
    case 2 => "A couple things here"
    case 3 | 4 => "A few things here" // match on multiple patterns
    case n if n < 0 => "Less than zero things here!?" // with a guard
    case _ => "Many things here" // a default case
}

numThings: Int = 3
res1_1: String = "A few things here"

## Inductive data structures
Pattern matching is particularly useful for interacting with inductive data structures.
Consider the following example, a list of integers.  (This is similar to how `List`s are implemented in Scala.)

$$\begin{array}{ccccc}
\textbf{NumList} & \rightarrow & Empty &\ |\  & Cons(\textbf{Num}, \textbf{NumList}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [3]:
sealed trait NumList

case object Empty extends NumList
case class Cons(hd: Int, tl: NumList) extends NumList

defined trait NumList
defined object Empty
defined class Cons

In [4]:
// example lists

// using the :: (cons operator) and Nil cases
val scalaList = 131 :: 3155 :: Nil
// this is equivalent to using the List constructor
scalaList == List(131, 3155)

// the same list using our NumList data structure instead of the built in one
val myNumList = Cons(131, Cons(3155, Empty))

scalaList: List[Int] = List(131, 3155)
res3_1: Boolean = true
myNumList: Cons = Cons(131, Cons(3155, Empty))

### Expr
Now we will implement our own grammer.  For this exercise, consider a simple language representing arithmetic expressions.
Given the grammer below, write the corresponding Scala code (assume $\textbf{Num}$ can be represented by a Scala `Int` as in $\textbf{NumList}$).

$$\begin{array}{ccccccccc}
\textbf{Expr} & \rightarrow & Const(\textbf{Num}) &\ |\  & Plus(\textbf{Expr}, \textbf{Expr}) &\ |\  & Minus(\textbf{Expr}, \textbf{Expr}) &\ |\  & Times(\textbf{Expr}, \textbf{Expr}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [5]:
// Begin Solution
sealed trait Expr

case class Const(n: Int) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Times(e1: Expr, e2: Expr) extends Expr
// end solution

defined trait Expr
defined class Const
defined class Plus
defined class Minus
defined class Times

In [6]:
// expressions using our new grammer
val myExpr0 = Const(0)
val myExpr1 = Plus(Const(500), Const(131)) // 500 + 131
val myExpr2 = Times(myExpr1, Minus(Const(105), Const(100))) // (500+131)*(105-100)

myExpr0: Const = Const(0)
myExpr1: Plus = Plus(Const(500), Const(131))
myExpr2: Times = Times(
  Plus(Const(500), Const(131)),
  Minus(Const(105), Const(100))
)

In [7]:
// Come up with your own example
val myExpr3 = ??? // Const(42)

myExpr3: Const = Const(42)

Now that we have our implementation of the grammer, we can write functions using this implementation.
We will often want to pattern match on the different cases of our grammer when writing these functions, as in the example below.

In [8]:
// Get the constants in an expression
def constVals(e: Expr): List[Int] = {
    e match {
        case Const(n) => n :: Nil // or List(n)
        case Plus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Minus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Times(e1, e2) => constVals(e1) ++ constVals(e2)
    }
}

defined function constVals

In [9]:
constVals(myExpr0)
constVals(myExpr2)
constVals(myExpr3)

res8_0: List[Int] = List(0)
res8_1: List[Int] = List(500, 131, 105, 100)
res8_2: List[Int] = List(42)

## Eval
Given an expression, can we compute the corresponding value?
Can we write a function to compute the value (evaluate the expression)?

In [10]:
def eval(e: Expr): Int = {
    // Begin Solution
    e match {
        case Const(n) => n
        case Plus (e1, e2) => eval(e1) + eval(e2)
        case Minus(e1, e2) => eval(e1) - eval(e2)
        case Times(e1, e2) => eval(e1) * eval(e2)
    }
    // End Solution
}

defined function eval

In [11]:
// test cases
eval(Const(42)) == 42
eval(myExpr0) == 0 // 0
eval(myExpr1) == 631 // 500 + 131
eval(myExpr2) == 3155 // (500+131)*(105-100)
eval(myExpr3)

res10_0: Boolean = true
res10_1: Boolean = true
res10_2: Boolean = true
res10_3: Boolean = true
res10_4: Int = 42

### Bonus

How would you add division or identifiers/variables to the Expr trait?

We will look at this more in the future, but thinking through this is a good way to practice.
(Hint: start by expanding the grammar and adding another case for the Expr trait.)